<a href="https://colab.research.google.com/github/commutant/___project/blob/main/catboost_cross_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Мой канал в Telegram: https://t.me/mikhaylovgleb  
Мой курс по DS на Stepik: https://stepik.org/z/113596  

# Загрузка данных

https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes?select=bmw.csv

https://drive.google.com/file/d/10pq7rvk_FtY-ulQ8ldVM4QAtaR6LWt-j/view?usp=sharing

In [ ]:
! gdown 10pq7rvk_FtY-ulQ8ldVM4QAtaR6LWt-j

Downloading...
From: https://drive.google.com/uc?id=10pq7rvk_FtY-ulQ8ldVM4QAtaR6LWt-j
To: /content/uk-used-cars.zip
100% 1.15M/1.15M [00:00<00:00, 74.1MB/s]


In [ ]:
! unzip -o /content/uk-used-cars.zip -d /content/uk-used-cars

Archive:  /content/uk-used-cars.zip
  inflating: /content/uk-used-cars/audi.csv  
  inflating: /content/uk-used-cars/bmw.csv  
  inflating: /content/uk-used-cars/cclass.csv  
  inflating: /content/uk-used-cars/focus.csv  
  inflating: /content/uk-used-cars/ford.csv  
  inflating: /content/uk-used-cars/hyundi.csv  
  inflating: /content/uk-used-cars/merc.csv  
  inflating: /content/uk-used-cars/skoda.csv  
  inflating: /content/uk-used-cars/toyota.csv  
  inflating: /content/uk-used-cars/unclean cclass.csv  
  inflating: /content/uk-used-cars/unclean focus.csv  
  inflating: /content/uk-used-cars/vauxhall.csv  
  inflating: /content/uk-used-cars/vw.csv  


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/uk-used-cars/bmw.csv')

In [ ]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


# Обучающая и тестовая выборка (train and test)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df,train_size=0.8,random_state=2007)

# Список фичей

In [ ]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

# Создание pool

In [ ]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from catboost import CatBoostRegressor
from catboost import cv, Pool

In [ ]:
train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                 )

In [ ]:
test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                 )

# Запуск кросс-валидации

In [ ]:
params = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
          'loss_function': 'RMSE',
              'learning_rate': 0.15,
              'random_seed': 2007,
              'verbose':100}

In [ ]:
cv_data = cv(
    params = params,
    pool = train_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=False,
    verbose=False,
    early_stopping_rounds=200
)

Training on fold [0/5]

bestTest = 0.07036770687
bestIteration = 997

Training on fold [1/5]

bestTest = 0.07224770421
bestIteration = 941

Training on fold [2/5]

bestTest = 0.07013407207
bestIteration = 996

Training on fold [3/5]

bestTest = 0.06594026995
bestIteration = 982

Training on fold [4/5]

bestTest = 0.07070219451
bestIteration = 855



# Находим лучшую итерацию

В таблице cv_data показаны средние значения метрик по количеству фолдов, которые мы выбрали. В данном случае 5.

In [ ]:
cv_data.head()

,iterations,test-MAPE-mean,test-MAPE-std,train-MAPE-mean,train-MAPE-std,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,0.839187,0.001779,0.839158,0.000388,21967.898000,390.041470,21972.008517,91.539899
1,1,0.706861,0.003608,0.705704,0.000509,19026.464933,418.923499,19007.118838,89.010685
2,2,0.597062,0.002519,0.595808,0.000624,16497.102588,415.611226,16477.325765,94.699757
3,3,0.505604,0.002335,0.504269,0.000677,14363.803100,456.411163,14340.134011,86.063035
4,4,0.428826,0.003038,0.427352,0.000984,12611.672046,470.647921,12588.175126,99.322102


In [ ]:
cv_data[cv_data['test-MAPE-mean'] == cv_data['test-MAPE-mean'].min()]

,iterations,test-MAPE-mean,test-MAPE-std,train-MAPE-mean,train-MAPE-std,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
988,988,0.069925,0.002361,0.051931,0.000853,2603.915598,503.315203,1583.325775,57.558281


In [ ]:
n_iters = cv_data[cv_data['test-MAPE-mean'] == cv_data['test-MAPE-mean'].min()]['iterations'].values[0]

In [ ]:
n_iters

988

Если мы подобрали learning_rate таким образом (чтобы его подобрать позапускай кросс-валидацию несколько раз меняя learning_rate, но скорее всего он и сам нормально подберется), что лучшая интерация случается ближе к 1000, то ее можно и не запоминать, а просто обучать модель на дефултной 1000 итераций с тем learning_rate который мы нашли

# Обучаемся на всех данных

Берем параметры, с которыми кросс-валидация дала желаемый результат. В моем случае лучшая итерация получилась ближе к 1000 с learning_rate = 0.15, поэтому я обучусь на дефултной 1000, т.е. вообще этот параметр указывать не буду.

In [ ]:
params = {'cat_features': cat_features,
          'eval_metric': 'MAPE',
          'loss_function': 'RMSE',
          'learning_rate': 0.15,
          'random_seed': 2007,
          'verbose':100}

In [ ]:
model = CatBoostRegressor(**params)

In [ ]:
model.fit(train_data)

0:	learn: 0.4258924	total: 9.25ms	remaining: 9.24s
100:	learn: 0.0758332	total: 942ms	remaining: 8.39s
200:	learn: 0.0679419	total: 1.9s	remaining: 7.55s
300:	learn: 0.0642885	total: 2.77s	remaining: 6.45s
400:	learn: 0.0617533	total: 3.55s	remaining: 5.3s
500:	learn: 0.0597092	total: 4.41s	remaining: 4.39s
600:	learn: 0.0581242	total: 6.01s	remaining: 3.99s
700:	learn: 0.0565338	total: 7.9s	remaining: 3.37s
800:	learn: 0.0549737	total: 9.83s	remaining: 2.44s
900:	learn: 0.0537485	total: 11.9s	remaining: 1.3s
999:	learn: 0.0526437	total: 14.2s	remaining: 0us


# Делаем предикт на тесте

In [ ]:
test['y_pred'] = model.predict(test_data)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
mean_absolute_percentage_error(test['price'],test['y_pred'])

0.07006576840330746

На кросс-валидации у нас где-то столько же и было, значит все ок:

In [ ]:
cv_data[cv_data['test-MAPE-mean'] == cv_data['test-MAPE-mean'].min()]

,iterations,test-MAPE-mean,test-MAPE-std,train-MAPE-mean,train-MAPE-std,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
988,988,0.069925,0.002361,0.051931,0.000853,2603.915598,503.315203,1583.325775,57.558281


Мой канал в Telegram: https://t.me/mikhaylovgleb  
Мой курс по DS на Stepik: https://stepik.org/z/113596  